There were 122K PPP records that had NAICS Code = 813110 (religious businesses).

These were geocoded by Google, to add lat/long and county name to them.

This notebook will take these records and add:
1. Census tract numbers
1. FIPS numbers
1. Congressional Districts

The result will be saved as a CSV file.

In [80]:
import folium
import geopandas as gpd
import pandas as pd

In [81]:
# to geocode the data with Google, I broke the file into 24 smaller chunks, each saved as a JSON file.
# the following lines read the JSONs and create the df
import glob
workDir = '/home/alp/Google Drive/Python/analysis/PPP_analysis/'

file_names = sorted(glob.glob(workDir+'*.done'))
firstFile = 're_df_chunk0.json.done'
df = pd.read_json(f"{workDir}{firstFile}")

for file in file_names:
    oldDF = df
    # print(file)
    if firstFile not in file:
        newDF = pd.read_json(file)
        df = pd.concat([oldDF,newDF])

In [82]:
df.head(3)

,LoanNumber,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,lat,lng,county,BorrowerZip,CurrentApprovalAmount,JobsReported,NAICSCode,Race,Ethnicity,BusinessType,ForgivenessAmount
0,9794577700,FRUIT COVE BAPTIST CHURCH OF JACKSONVILLE FL INC,501 State Road 13,Saint Johns,None,30.116637,-81.625214,St. Johns County,32259-2832,289765.00,89.0,813110,Unanswered,Unknown/NotStated,Non-Profit Organization,291741.75
1,8414257104,CORPORATION OF THE CATHOLIC ARCHBISHOP OF ANCH...,225 Cordova Street,ANCHORAGE,AK,61.220191,-149.878792,Anchorage Municipality,99501,1643548.82,214.0,813110,Unanswered,Unknown/NotStated,Non-Profit Organization,1660524.65
2,9828357104,"ANCHORAGE BAPTIST TEMPLE, INC.",6401 E Northern Lights Blvd,ANCHORAGE,AK,61.196319,-149.760805,Anchorage Municipality,99504-3312,961962.29,166.0,813110,Unanswered,Unknown/NotStated,Non-Profit Organization,969895.18


To add the new datafields, we will be doing a join between this df (with religion businesses) and the GeoPandasDFs with census tracts and FIPS data.

To do this, we will use the GeoPandas .sjoin function to look at each lat/long point in the religion df and match it to the different regions defined in the other DFs. That is, they have a "geometry" field that defines a geographic region; GeoPandas allows us to figure out which "geometry" contains the point. We can use this to add fields from the other tables to the Religion table.

Steps:
1. Create a POINTS column with geopandas point data using the geo-encoding standard
1. Convert the religion df to a geopandas df
1. Load in the Census tract data, which we will use to add the tract number and the FIPS number

In [83]:
df['points'] = gpd.points_from_xy(df.lng, df.lat,crs="EPSG:4326")
gdf = gpd.GeoDataFrame(data=df,geometry='points')
gdf.crs


<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Now we will 
1. get the census tract information
1. create a df with just the tract number, FIPS number, and geometry
1. convert it to the same geo-encoding standard

In [84]:
c_tracts = gpd.read_file("/home/alp/Google Drive/Python/GeoShapeFiles/cb_2021_us_tract_500k.zip")

In [85]:
c_tracts.head(3)

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,13,163,960300,1400000US13163960300,13163960300,9603,Census Tract 9603,GA,Jefferson County,Georgia,CT,564728206,4845400,"POLYGON ((-82.66192 33.12633, -82.65194 33.126..."
1,37,109,071201,1400000US37109071201,37109071201,712.01,Census Tract 712.01,NC,Lincoln County,North Carolina,CT,5976613,6611763,"POLYGON ((-80.99344 35.47878, -80.99142 35.480..."
2,25,025,050200,1400000US25025050200,25025050200,502,Census Tract 502,MA,Suffolk County,Massachusetts,CT,243271,0,"POLYGON ((-71.04003 42.37865, -71.03921 42.378..."


We need to combine the STATEF and COUNTYFP fields to create the FIPS number that is used by a lot of other tables. (We can pull the FIPS number apart, if, for some reason, we need just the state number.)

In [86]:
c_tracts['FIPS'] = (c_tracts['STATEFP']+c_tracts['COUNTYFP']).astype(str)


The census tract table has a lot of data we know already (state, county) or don't care about (area that is land, area that is water). We will drop all these and just get the two columns with data we care about: census tract numbers and geometry.

In [87]:
c_tracts = c_tracts[['TRACTCE','FIPS','geometry']]

In [88]:
c_tracts_EPSG = c_tracts.to_crs('EPSG:4326')

Create a new geopandas df by doing an .sjoin() that will match the point data from the religion DF to the geometry data in teh census tracts.

This will result in census tract numbers being assigned to all the rows in the religion df.

In [89]:
df2 = gpd.sjoin(left_df = gdf,right_df=c_tracts_EPSG,how='left')
df2.head(3)

,LoanNumber,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,lat,lng,county,BorrowerZip,CurrentApprovalAmount,JobsReported,NAICSCode,Race,Ethnicity,BusinessType,ForgivenessAmount,points,index_right,TRACTCE,FIPS
0,9794577700,FRUIT COVE BAPTIST CHURCH OF JACKSONVILLE FL INC,501 State Road 13,Saint Johns,None,30.116637,-81.625214,St. Johns County,32259-2832,289765.00,89.0,813110,Unanswered,Unknown/NotStated,Non-Profit Organization,291741.75,POINT (-81.62521 30.11664),45671.0,020802,12109
1,8414257104,CORPORATION OF THE CATHOLIC ARCHBISHOP OF ANCH...,225 Cordova Street,ANCHORAGE,AK,61.220191,-149.878792,Anchorage Municipality,99501,1643548.82,214.0,813110,Unanswered,Unknown/NotStated,Non-Profit Organization,1660524.65,POINT (-149.87879 61.22019),78940.0,000601,02020
2,9828357104,"ANCHORAGE BAPTIST TEMPLE, INC.",6401 E Northern Lights Blvd,ANCHORAGE,AK,61.196319,-149.760805,Anchorage Municipality,99504-3312,961962.29,166.0,813110,Unanswered,Unknown/NotStated,Non-Profit Organization,969895.18,POINT (-149.76081 61.19632),32590.0,001702,02020


To make sure the .sjoin() matched the FIPS number AND the TRACTCE number, let's look at one FIPS number (that is, a county) and see how many tracts it contains.

In [90]:
df2[df2['FIPS'] == '02020']['TRACTCE'].value_counts()

001500    10
002812     8
001000     8
001900     7
002501     6
000702     5
000102     4
002811     4
001100     4
001200     4
000201     4
002712     3
001300     3
001731     3
001801     3
001702     3
002900     2
002713     2
002711     2
001601     2
000601     2
000202     2
002601     2
002502     1
002822     1
002000     1
000500     1
000801     1
002821     1
002302     1
000701     1
002714     1
002201     1
000206     1
001401     1
Name: TRACTCE, dtype: int64

In [91]:
len(pd.unique(df2['TRACTCE']))

19203

Interesting, of the 85K census tracts, only 19K had borrowers in them.

How many records lack FIPS and/or TRACTCE numbers?

In [92]:
len(df2[df2['FIPS'].isna()])

37

There are only 37 records that lack data -- mostly because it seems Google didn't find lat/lon data for them.

Given this small number, this would be a good job for a student researcher -- find the addresses on Google maps and find the lat/long data for them.

I'll delete the unneeded columns ('POINT' and 'index_right').

In [93]:
df2 = df2.drop(columns = ['points','index_right'])
df2.columns

Index(['LoanNumber', 'BorrowerName', 'BorrowerAddress', 'BorrowerCity',
       'BorrowerState', 'lat', 'lng', 'county', 'BorrowerZip',
       'CurrentApprovalAmount', 'JobsReported', 'NAICSCode', 'Race',
       'Ethnicity', 'BusinessType', 'ForgivenessAmount', 'TRACTCE', 'FIPS'],
      dtype='object')

Finally, I didn't keep the Congressional districts from the original data, so we will add that data back. Since the LoanNumber is a unique identifier, all I need to do is to create a DF from the original data that has the loan number and CD, and then join that with the df I've been adding information to.

In [94]:
import glob
workDir = '/home/alp/Google Drive/Python/analysis/PPP_analysis/data/'

file_names = sorted(glob.glob(workDir+'public_up_to*.csv'))
firstFile = 'public_150k_plus_220930.csv'


#colList = ['LoanNumber','CD','BorrowerZip','ProjectZip','ProjectCountyName',
 #           'JobsReported','NAICSCode',
 #           'Race','Ethnicity','Gender','Veteran',
 #           'BusinessType','ForgivenessAmount']

colList = ['LoanNumber','CD',]

cd_df = pd.read_csv(workDir+firstFile, usecols = colList)
#df = df[df['NAICSCode'] == 813110]

for file in file_names:
    oldDF = cd_df
    #print(file)
    if firstFile not in file:
        newDF = pd.read_csv(file, usecols = colList)
        #newDF = newDF[newDF['NAICSCode'] == 813110]
        cd_df = pd.concat([oldDF,newDF])

In [110]:
final = pd.merge(left=df2,right=cd_df,left_on='LoanNumber',right_on='LoanNumber',how='left')

In [111]:
final[final['BorrowerState'] == 'DE']['CD'].value_counts()

DE-    329
Name: CD, dtype: int64

There are seven states (+DC) that have only 1 rep (AK, DE, MT, ND, SD, VT, WY). Currently, their CD is listed as the state abbreviation + dash (DE-).

We need to change this so they have an 01 apppended (DE-01), so they will match the rest of the data.

The data also has records for PR, GU, VI, etc. We'll keep all those, but also add the '01' to them. 

In [112]:
single_rep_list = final[final['CD'].str.len() == 3]['CD'].value_counts().index.to_list()
single_rep_list

['SD-',
 'ND-',
 'PR-',
 'DC-',
 'MT-',
 'DE-',
 'VT-',
 'AK-',
 'WY-',
 'GU-',
 'VI-',
 'MP-',
 'AS-',
 'AE-',
 'NC-']

In [113]:
final[final['CD']=='NC-']

,LoanNumber,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,lat,lng,county,BorrowerZip,CurrentApprovalAmount,JobsReported,NAICSCode,Race,Ethnicity,BusinessType,ForgivenessAmount,TRACTCE,FIPS,CD
66810,4363977708,ST JOSEPH OF THE HILLS CATHOLIC CHURCH,316 BOONE RD,EDEN,NC,36.499757,-79.759111,Rockingham County,28288,13612.0,1.0,813110,Unanswered,Unknown/NotStated,Non-Profit Organization,13710.45,040300,37157,NC-


Let's fix this manually. NC's districts are a mess, as the state house keeps re-drawing lines, which are struck down by the NC supreme court.

After some research, I think Eden, NC is in the 6th district. 

In [114]:
final.loc[final['CD'] == 'NC-', 'CD'] = "NC-06"

In [115]:
single_rep_list = final[final['CD'].str.len() == 3]['CD'].value_counts().index.to_list()
single_rep_list

['SD-',
 'ND-',
 'PR-',
 'DC-',
 'MT-',
 'DE-',
 'VT-',
 'AK-',
 'WY-',
 'GU-',
 'VI-',
 'MP-',
 'AS-',
 'AE-']

In [116]:
#https://www.geeksforgeeks.org/how-to-replace-values-in-column-based-on-condition-in-pandas/
#Syntax: df.loc[ df[“column_name”] == “some_value”, “column_name”] = “value”

for state in single_rep_list:
    final.loc[final['CD'] == state, 'CD'] = f"{state}01"


In [117]:
final[final['BorrowerState'] == 'DE']['CD'].value_counts()

DE-01    329
Name: CD, dtype: int64

In [118]:
# DC is (of course) treated differently -- it shows up in the censs records as DC-99
final.loc[final['CD'] == 'DC-01', 'CD'] = 'DC-99'

Let's see if there are other messed up CDs. . .

In [119]:
final[final['CD'].str.len() == 3]['CD'].value_counts()

Series([], Name: CD, dtype: int64)

In [120]:
len(final[final['CD'].isna()])

6

In [121]:
final[final['CD'].isna()]

,LoanNumber,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,lat,lng,county,BorrowerZip,CurrentApprovalAmount,JobsReported,NAICSCode,Race,Ethnicity,BusinessType,ForgivenessAmount,TRACTCE,FIPS,CD
30118,8169597706,GRACE EVANGELICAL LUTHERAN CHURCH INC,3434 6th Street,Lewiston,ID,46.377232,-117.002555,Nez Perce County,83504,18800.0,5.0,813110,Unanswered,Unknown/NotStated,Non-Profit Organization,18887.05,960700,16069,NaN
82803,7598347705,LIGHTHOUSE MINISTRIES,464 Rathmell Rd,Lockbourne,OH,39.863566,-82.994756,Franklin County,43237,14000.0,4.0,813110,Unanswered,Unknown/NotStated,Non-Profit Organization,14139.22,009590,39049,NaN
110104,9368327703,"NATIONAL ASSOCIATION OF CATHOLIC CHAPLAINS, LTD.",4915 S HOWELL AVE SUITE 501,MILWAUKEE,WI,42.955234,-87.910435,Milwaukee County,54320,106647.5,7.0,813110,Unanswered,Unknown/NotStated,Limited Liability Company(LLC),107687.68,021600,55079,NaN
111562,9302477710,FIRST CONGREGATIONAL CHURCH OF RIPON,220 RANSOM ST,RIPON,WI,43.844855,-88.840666,Fond du Lac County,57971,18500.0,6.0,813110,Unanswered,Unknown/NotStated,Limited Liability Company(LLC),18683.48,041600,55039,NaN
113591,9772027709,HOLY TRINITY BAPTIST CHURCH,None,None,None,-999.000000,-999.000000,Not Found,None,32625.0,3.0,813110,Unanswered,Unknown/NotStated,None,32891.36,NaN,NaN,NaN
113592,9727847709,RHEMA EMPOWERMENT CENTER CHURCH,None,None,None,-999.000000,-999.000000,Not Found,None,9000.0,1.0,813110,Unanswered,Unknown/NotStated,Professional Association,9099.37,NaN,NaN,NaN


In [73]:
final.loc[final['LoanNumber'] == '8169597706', 'CD'] = 'ID-01'
# 83504 is an invalid ZIP. It should be 83501. According to https://ziplook.house.gov/htbin/findrep_house that puts it in ID-01 
final.loc[final['LoanNumber'] == '7598347705', 'CD'] = 'ID-01'
# 43237 is an invalid ZIP. It should be 43137. > OH-15 (Not sure this is correct, b/c of the new district maps)
final.loc[final['LoanNumber'] == '9368327703', 'CD'] = 'WI-04'
# 54230 invalid ZIP. Should be 53207. > WI-04
final.loc[final['LoanNumber'] == '9302477710', 'CD'] = 'WI-06'
# 54971 IS a valid ZIP! > WI-06

In [125]:
final[final['CD'].isna()]

,LoanNumber,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,lat,lng,county,BorrowerZip,CurrentApprovalAmount,JobsReported,NAICSCode,Race,Ethnicity,BusinessType,ForgivenessAmount,TRACTCE,FIPS,CD
30118,8169597706,GRACE EVANGELICAL LUTHERAN CHURCH INC,3434 6th Street,Lewiston,ID,46.377232,-117.002555,Nez Perce County,83504,18800.0,5.0,813110,Unanswered,Unknown/NotStated,Non-Profit Organization,18887.05,960700,16069,NaN
82803,7598347705,LIGHTHOUSE MINISTRIES,464 Rathmell Rd,Lockbourne,OH,39.863566,-82.994756,Franklin County,43237,14000.0,4.0,813110,Unanswered,Unknown/NotStated,Non-Profit Organization,14139.22,009590,39049,NaN
110104,9368327703,"NATIONAL ASSOCIATION OF CATHOLIC CHAPLAINS, LTD.",4915 S HOWELL AVE SUITE 501,MILWAUKEE,WI,42.955234,-87.910435,Milwaukee County,54320,106647.5,7.0,813110,Unanswered,Unknown/NotStated,Limited Liability Company(LLC),107687.68,021600,55079,NaN
111562,9302477710,FIRST CONGREGATIONAL CHURCH OF RIPON,220 RANSOM ST,RIPON,WI,43.844855,-88.840666,Fond du Lac County,57971,18500.0,6.0,813110,Unanswered,Unknown/NotStated,Limited Liability Company(LLC),18683.48,041600,55039,NaN
113591,9772027709,HOLY TRINITY BAPTIST CHURCH,None,None,None,-999.000000,-999.000000,Not Found,None,32625.0,3.0,813110,Unanswered,Unknown/NotStated,None,32891.36,NaN,NaN,NaN
113592,9727847709,RHEMA EMPOWERMENT CENTER CHURCH,None,None,None,-999.000000,-999.000000,Not Found,None,9000.0,1.0,813110,Unanswered,Unknown/NotStated,Professional Association,9099.37,NaN,NaN,NaN


Annoying! Apparently, I can't search for a row by COL A and change a value in COL B.

Well, let's change all the NAN values to ='-99999' so we can find them easily in Excel.
Then we will save df as a TAB delimited CSV. (I'm using TAB because there are some commas in some of the names, which screws up the CSV import.)

In [ ]:
#https://stackoverflow.com/questions/31511997/pandas-dataframe-replace-all-values-in-a-column-based-on-condition
import numpy as np
final['CD'] = np.where(final['LoanNumber'] == '8169597706','ID-01',final['CD'])

According to the link above, I should be able to do this with numpy, but it doesn't work either.

In [75]:
final.columns

Index(['LoanNumber', 'BorrowerName', 'BorrowerAddress', 'BorrowerCity',
       'BorrowerState', 'lat', 'lng', 'county', 'BorrowerZip',
       'CurrentApprovalAmount', 'JobsReported', 'NAICSCode', 'Race',
       'Ethnicity', 'BusinessType', 'ForgivenessAmount', 'TRACTCE', 'FIPS',
       'CD'],
      dtype='object')

In [76]:
final['CD'] = final['CD'].fillna('-9999')

In [78]:
final['FIPS'] = final['FIPS'].fillna('-9999')
final['TRACTCE'] = final['TRACTCE'].fillna('-9999')

In [79]:
final.to_csv('PPP_RE_with_FIPS_CD_CensusT.csv',sep = '\t')